# 00 - Clean files

In [21]:
!pip install pandas pyarrow liac-arff fastparquet


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Hurtlex

In [22]:
import os
import pandas as pd

In [23]:
# Configurações iniciais
lexic_file = '../data/01-raw/HurtLex-PT/hurtlex_PT.tsv'

# Criar o DataFrame Pandas
raw_df = pd.read_csv(lexic_file, sep='\t')

# Separar os conservatives
conservative_df = raw_df[raw_df['level'] == 'conservative']

# Salvar como csv
new_path = '../data/02-cleaned/'
os.makedirs(os.path.dirname(new_path), exist_ok=True)

file_name = 'hurtlex_PT_conservatives.csv'
conservative_df.to_csv(new_path + file_name, index=False)

## Funções de Limpeza de Dataset

In [24]:
import arff, csv
from typing import List

In [27]:
# Cria um objeto dataset, para armazenar as informações necessárias
class Dataset:
	def __init__(self, name: str, orig_path: str | List[str]):
		self.name = name
		self.orig_path = orig_path
		# Normaliza para sempre ser uma lista, facilitando a iteração
		self.paths = orig_path if isinstance(orig_path, list) else [orig_path]

		self.output_path = f'../data/02-cleaned/{name.lower( )}.csv'
		self.df = self.get_df( )

		print(f"Dataset '{name}' criado com sucesso ({len(self.paths)} arquivos, com {self.df.shape[0]} registros)!")

	def get_df(self):
		temp_dfs = []

		for path in self.paths:
			# Verifica a extensão para escolher o leitor correto
			if path.endswith('.csv'):
				# Adicione sep= ou encoding= aqui se necessário para datasets específicos
				df = pd.read_csv('../data/01-raw/Datasets/' + path)

			elif path.endswith('.arff'):
				with open('../data/01-raw/Datasets/' + path, 'r') as f:
					obj = arff.load(f)
					data = obj['data']
					# Extrai o nome dos atributos para usar como colunas
					attrs = [attr[0] for attr in obj['attributes']]
					df = pd.DataFrame(data, columns=attrs)

			elif path.endswith('.parquet'):
				# Caso precise ler parquet no futuro (ex: OLID original)
				df = pd.read_parquet('../data/01-raw/Datasets/' + path, engine='fastparquet')

			else:
				raise ValueError(f"Formato de arquivo não suportado: {path}")

			temp_dfs.append(df)

		# Se houver mais de um arquivo (ex: treino e teste), concatena um embaixo do outro
		if len(temp_dfs) > 1:
			return pd.concat(temp_dfs, ignore_index=True)

		return temp_dfs[0]


In [28]:
# Objetos para cada dataset
fortuna_obj = Dataset('Fortuna', 'Fortuna/2019-05-28_portuguese_hate_speech_binary_classification.csv')
hatebrxplain_obj = Dataset('HateBRXplain', 'HateBRXplain/dataset/HateBRXplain/HateBRXplain.csv')
offcombr3_obj = Dataset('OffComBR3', 'OffComBR/OffComBR3.arff')
olidbr_obj = Dataset('OLID-BR', ['OLID-BR/test.csv', 'OLID-BR/train.csv'])
toldbr_obj = Dataset('ToLD-BR', 'ToLD-BR/ToLD-BR.csv')
tupy_obj = Dataset('TuPy', 'TuPy/binary/tupy_binary_vote.csv')

# Lista de todos os objetos
datasets_obj = [fortuna_obj, hatebrxplain_obj, offcombr3_obj, olidbr_obj, toldbr_obj, tupy_obj]

Dataset 'Fortuna' criado com sucesso (1 arquivos, com 5670 registros)!
Dataset 'HateBRXplain' criado com sucesso (1 arquivos, com 7000 registros)!
Dataset 'OffComBR3' criado com sucesso (1 arquivos, com 1033 registros)!
Dataset 'OLID-BR' criado com sucesso (2 arquivos, com 6952 registros)!
Dataset 'ToLD-BR' criado com sucesso (1 arquivos, com 21000 registros)!
Dataset 'TuPy' criado com sucesso (1 arquivos, com 10000 registros)!


In [29]:
def get_df_columns(df: pd.DataFrame, columns_to_keep: list, has_to_calc_toxic: bool, columns_to_calc: list, columns_name: list):

	# Se a coluna de toxicidade ainda precisa ser "calculada", cria uma nova coluna com 1 se alguma das columns_to_calc considerar toxico (for 1 ou mais), e 0 se nenhum.
	if has_to_calc_toxic:
		df['is_toxic'] = df[columns_to_calc].max(axis=1)
		print(f"# --- Nova coluna criada: \'is_toxic\'")

		# Adiciona a nova coluna às colunas desejadas
		columns_to_keep.append('is_toxic')

	# Faz o recorte do DataFrame para as colunas desejadas e as renomeia para o padrão.
	df = df[columns_to_keep]
	print(f"# --- Colunas selecionadas: {df.columns}")
	df.columns = columns_name
	print(f"# --- Colunas renomeadas: {df.columns}")

	return df

def get_normalized_toxic_col(df: pd.DataFrame, mapping: dict):
	print(f"# --- Coluna 'is_toxic' normalizada para booleano")
	return df['is_toxic'].map(mapping)

def get_normalized_text_col(df: pd.DataFrame):
	# Retira menções ao longo do texto, se antes do @ não for uma letra/palavra, e substitui por @user
	df['text'] = df['text'].str.replace(r'(?<!\w)@\w+', '@user', regex=True)
	df['text'] = df['text'].str.replace(r'\bUSER\b', '@user', regex=True)
	print(f"# --- Menções normalizadas para '@user'")

	# Retira URLs ao longo do texto
	df['text'] = df['text'].str.replace(r'(?:https?://|www\.)\S+', '<url>', regex=True)
	df['text'] = df['text'].str.replace(r'(?i)\blink\b(?=(?:\s*link\b)*\s*$)', '<url>', regex=True)
	df['text'] = df['text'].str.replace(r'\bURL\b', '<url>', regex=True)
	print(f"# --- Links normalizados para '<url>'")

	# Normaliza os retweets, transformando para <RT>
	df['text'] = df['text'].str.replace(r'\bRT\b', '<RT>', regex=True)
	df['text'] = df['text'].str.replace(r'\brt\b', '<RT>', regex=True)
	print(f"# --- Retweets normalizados para '<RT>'")

	# Retira '\n' e substitui por ' '
	df['text'] = df['text'].str.replace('\n', ' ', regex=False)
	print(f"# --- \'\\n\' normalizado para ' '")

	return df['text']

def remove_unexpressive_data(df: pd.DataFrame):
	# Desconsidera todos os is_toxic sem valor (None)
	df = df[df['is_toxic'].notna( )]
	print(f"#\n# --- Registros com is_toxic=None removidos")
	print(f"# Quantidade de registros após remoção de valores incompletos: {df.shape[0]}")

	# Desconsidera entradas duplicadas
	df = df.drop_duplicates(subset='text', keep='first')
	print(f"#\n# --- Registros duplicados removidos")
	print(f"# Quantidade de registros após remoção de duplicatas: {df.shape[0]}")

	return df

def save_final_df(dataset: Dataset):
	# Salva o DataFrame final como CSV
	dataset.df.to_csv(dataset.output_path, index=False, quoting=csv.QUOTE_NONNUMERIC)
	print(f"#\n# Salvo em: {dataset.output_path}")

# Função principal de limpeza de um dataset
def clean_dataset(dataset: Dataset, columns_to_keep: list,
				  obs: str = None, has_to_calc_toxic: bool = False, columns_to_calc: list = None, columns_name: list=None,
				  toxic_mapping: dict=None
				  ):

	if toxic_mapping is None:
		toxic_mapping = {'1':True, '0':False}
	if columns_name is None:
		columns_name = ['text', 'is_toxic']

	print(f"\n########## LIMPEZA DE DADOS: {dataset.name}")

	# Escreve alguma observação sobre o dataset, se desejado
	if obs is not None:
		print(f"# Observação: {obs}\n#\n")

	# Faz uma cópia do df
	df = dataset.df.copy( )

	# Escreve o número de registros do dataset
	print(f"# Número de registros (raw): {df.shape[0]}")

	# Pega as colunas desejadas e padroniza os nomes
	df = get_df_columns(df=df, columns_to_keep=columns_to_keep, has_to_calc_toxic=has_to_calc_toxic, columns_to_calc=columns_to_calc, columns_name=columns_name)

	# Normaliza a coluna 'is_toxic'
	df['is_toxic'] = get_normalized_toxic_col(df=df, mapping=toxic_mapping)

	# Normaliza a coluna 'text'
	df['text'] = get_normalized_text_col(df=df)

	# Remove dados duplicados ou sem anotação
	df = remove_unexpressive_data(df=df)

	# Atualiza o df no dataset
	dataset.df = df

	# Salva o df final
	save_final_df(dataset=dataset)



In [30]:
clean_dataset(fortuna_obj, columns_to_keep = ['text', 'hatespeech_comb'], 
              obs='(binary version)',
              toxic_mapping={1:True, 0:False})

clean_dataset(hatebrxplain_obj, columns_to_keep = ['comment', 'offensive_label'],
              toxic_mapping={1:True, 0:False})

clean_dataset(offcombr3_obj, columns_to_keep = ['document', '@@class'],
              toxic_mapping={'yes':True, 'no':False})

clean_dataset(olidbr_obj, columns_to_keep = ['text', 'is_offensive'],
              obs='(treino e teste)',
              toxic_mapping={'NOT':False, 'OFF':True})

clean_dataset(toldbr_obj, columns_to_keep=['text'], has_to_calc_toxic=True, 
              columns_to_calc=['homophobia', 'obscene', 'insult', 'racism', 'misogyny', 'xenophobia'], toxic_mapping={1:True, 0:False})

clean_dataset(tupy_obj, columns_to_keep = ['text', 'hate'], obs='(versão not-expanded)',
              toxic_mapping={1:True, 0:False})


########## LIMPEZA DE DADOS: Fortuna
# Observação: (binary version)
#

# Número de registros (raw): 5670
# --- Colunas selecionadas: Index(['text', 'hatespeech_comb'], dtype='object')
# --- Colunas renomeadas: Index(['text', 'is_toxic'], dtype='object')
# --- Coluna 'is_toxic' normalizada para booleano
# --- Menções normalizadas para '@user'
# --- Links normalizados para '<url>'
# --- Retweets normalizados para '<RT>'
# --- '\n' normalizado para ' '
#
# --- Registros com is_toxic=None removidos
# Quantidade de registros após remoção de valores incompletos: 5670
#
# --- Registros duplicados removidos
# Quantidade de registros após remoção de duplicatas: 5666
#
# Salvo em: ../data/02-cleaned/fortuna.csv

########## LIMPEZA DE DADOS: HateBRXplain
# Número de registros (raw): 7000
# --- Colunas selecionadas: Index(['comment', 'offensive_label'], dtype='object')
# --- Colunas renomeadas: Index(['text', 'is_toxic'], dtype='object')
# --- Coluna 'is_toxic' normalizada para booleano
# --- Men